# Import libraries

In [ ]:
import lyricsgenius
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


# Creating Database

## Dave database

In [ ]:

def list_songs_url80s(url):
    """
    Esta función recibe el URL asociado a la base de datos de dave del top 100 canciones en inglés unicamente de
    los 80, como string y devuelve una lista de las 100 canciones de esta decada.
    
    """
    headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    #Obtengo un set de los tags presentes en el tag 'p'
    tags = soup.find_all(name="p")
    
    #Separa en elementos de una lista las canciones.
    songs = ''.join(i.text for i in tags[12:22]).split('\n')
    
    #Elimina los espacios de los extremos de cada elemento de la lista si
    #los hubiese y devuelve una lista nueva con cada elemento
    songs = [i.strip() for i in songs]
    
    #Elimina los elementos vacios de la lista
    songs = list(filter(None, songs))
    return songs

#test

test_80 = 'https://davesmusicdatabase.blogspot.com/2014/08/top-100-songs-from-1980-to-1989.html'
list_songs_url80s(test_80)

In [ ]:
def list_songs_url(url):
    """
    Esta función recibe un URL asociado a las bases de datos de dave del top 100 canciones en inglés de cada
    década, como string y devuelve una lista de las 100 canciones de esta decada.
    Nota: no sirve para la década de los 80
    
    """
    headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    #Obtengo un set de los tags presentes en el tag 'p'
    tags = soup.find_all(name="p")
    
    #Separa en elementos de una lista las canciones.
    songs = ''.join(i.text for i in tags[5:15]).split('\n')
    
    #Elimina los espacios de los extremos de cada elemento de la lista si
    #los hubiese y devuelve una lista nueva con cada elemento
    songs = [i.strip() for i in songs]
    
    #Elimina los elementos vacios de la lista
    songs = list(filter(None, songs))
    return songs

#test
url_dave_30s = "https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-of-1930-1939.html"
list_songs_url(test_80)

In [ ]:
test_str = '81. The Peanut Vendor (El Manisero)...Don Azpiazu with Antonio Machin (1930)'

#Recibe un string y devuelve el a;o de el string de la cancion
def extract_year(song_string):
    return song_string[-5:-1]

extract_year(test_str)

In [ ]:
#Recibe un string y devuelve el a;o de el string de la cancion
def extract_decade(song_string):
    return extract_year(song_string)[:3] + '0'

extract_decade(test_str)

In [ ]:
#Recibe un string y devuelve el orden de la cancion
def extract_order(song_string):
    return song_string[:3].replace('.','').strip()

extract_order(test_str)

In [ ]:
#Recibe un string y devuelve el titulo de la cancion
def extract_title(song_string):
    title_1 = re.findall('\d+.\s(.*)(?:\.\.\.|\…)',song_string)
    
    if title_1:
        return title_1[0]
    
    title_2 = re.findall('“(.*?)”', song_string)
    
    if title_2:
        return title_2[0]
    
    return ''


#test
url_dave_70s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1970-1979.html?m=0'

list_70s = list_songs_url(url_dave_70s)
list_70s

for i in list_70s:
    song_70 = extract_title(i)
    print(song_70)

In [ ]:
#Recibe un string y devuelve el autor de la cancion
def extract_author(song_string):
    author_1 = re.findall('(?:\.\.\.|\…)(.*)\(',song_string)
    
    if author_1:
        return author_1[0]
    
    author_2 = re.findall('”\s(.*)\s\(',song_string)
    if author_2 == ' ':
        pass
    
    if author_2:
        return author_2[0]
    
    author_3 = re.findall('\.\s(.*)\s“',song_string)
    if author_3:
        return author_3[0]

#test
for i in list_70s:
    author_70 = extract_author(i)
    print(author_70)

In [ ]:
def ext_data_songs(url_dave):
    list_songs_dave = list_songs_url(url_dave)
    if list_songs_dave[0].endswith('Database.'):
        list_songs_dave = list_songs_url80s(url_dave)
        
    formated_songs = []
    for i in list_songs_dave:
        decade = extract_decade(i)
        year = extract_year(i)
        title = extract_title(i)
        author = extract_author(i)
        order = extract_order(i)
        
        song = {
            'decade':decade,
            'year':year,
            'title':title,
            'author':author,
            'order':order,
        }
        formated_songs.append(song)
    return formated_songs

ext_data_songs(test_80)

In [ ]:
url_dave_00s = 'https://davesmusicdatabase.blogspot.com/2017/10/the-top-100-songs-from-2000-2009.html'
url_dave_20s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-from-1920-1929.html'
url_dave_30s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-of-1930-1939.html'
url_dave_40s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-from-1940-1949.html'
url_dave_50s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-from-1950-to-1959.html'
url_dave_60s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-100-songs-from-1960-1969_2.html'
url_dave_70s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1970-1979.html?m=0'
url_dave_80s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-100-songs-from-1980-to-1989.html'
url_dave_90s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1990-to-1999.html'
url_dave_2010s = 'https://davesmusicdatabase.blogspot.com/2017/10/the-top-50-songs-from-2010-2016.html'

dicc_00s = ext_data_songs(url_dave_00s)
dicc_20s = ext_data_songs(url_dave_20s)
dicc_30s = ext_data_songs(url_dave_30s)
dicc_40s = ext_data_songs(url_dave_40s)
dicc_50s = ext_data_songs(url_dave_50s)
dicc_60s = ext_data_songs(url_dave_60s)
dicc_70s = ext_data_songs(url_dave_70s)
dicc_80s = ext_data_songs(url_dave_80s)
dicc_90s = ext_data_songs(url_dave_90s)
dicc_2010s = ext_data_songs(url_dave_2010s)

dicc_00s

In [ ]:
df_00s = pd.DataFrame(dicc_00s)
df_20s = pd.DataFrame(dicc_20s)
df_30s = pd.DataFrame(dicc_30s)
df_40s = pd.DataFrame(dicc_40s)
df_50s = pd.DataFrame(dicc_50s)
df_60s = pd.DataFrame(dicc_60s)
df_70s = pd.DataFrame(dicc_70s)
df_80s = pd.DataFrame(dicc_80s)
df_90s = pd.DataFrame(dicc_90s)
df_2010s = pd.DataFrame(dicc_2010s)

In [ ]:
df_dave_songs = pd.concat([df_00s, df_20s, df_30s, df_40s, df_50s, df_60s, df_70s, df_80s, df_90s, df_2010s], axis=0, join='inner')
df_dave_songs

In [ ]:
df_dave_songs.to_csv('/Users/carlasanchezvalles/ironhack/Proyectos/Proyecto_2/dave_songs.csv', index=False)


# Spotify

In [ ]:
def first_search_spoti(title, author):
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id="00e41fa4725d4469b29d9e36f43d7ec9",
        client_secret="e64b15d24085495283f18937da5452d3")
                        )
    search = sp.search(title + ' ' + author, market='US')
    
    tracks = search['tracks']
    
    df = pd.json_normalize(tracks['items'])
    
    df = df[['id', 'name', 'popularity', 'artists', 'album.name', 'external_urls.spotify', 'uri', 'explicit', 'album.release_date', 'duration_ms']]
    
    df['artists'] = df.artists.apply(lambda d: d[0]['name'])
    
    return df.head(1)

In [ ]:
def songs_to_spoti_df(songs_df):
    lista = []

    for i in range(len(songs_df)):
        try:
            search = first_search_spoti(songs_df.loc[i, "title"], songs_df.loc[i, "author"])
            lista.append(search)
        except KeyError:
            pass
    
    return pd.concat(lista, axis=0)


df_30s_spoti = songs_to_spoti_df(df_30s)

# Genius

In [ ]:
def get_song_lyrics(artist_name, song_name):
    
    genius = lyricsgenius.Genius(
        'v3s3cdBYSN4Kta3F2HDTmdqqNOckKEZTQA5O5Bhb5J-BV_L02jqPGCEnvpKsSrBz')
    
    artist = genius.search_artist(artist_name, max_songs=0, sort="title")
    
    song = artist.song(song_name)
    
    return re.sub("[\(\[].*?[\)\]]", "", song.lyrics.replace('\n', ' ')).lower().replace('lyrics','').replace('embed','')

In [ ]:
def add_lyrics_to_df(spoti_songs_df):
    
    genius = lyricsgenius.Genius(
        'v3s3cdBYSN4Kta3F2HDTmdqqNOckKEZTQA5O5Bhb5J-BV_L02jqPGCEnvpKsSrBz')
        
    lyrics_list = []

    for artist, song in zip(spoti_songs_df['artists'], spoti_songs_df['name']):
        try:
            lyrics_list.append(get_song_lyrics(artist, song))
        except AttributeError:
            lyrics_list.append('None')
    
    spoti_songs_df['lyrics'] = lyrics_list
    
    return spoti_songs_df

In [ ]:
def add_word_list_to_df(df_with_lyrics):
    
    genius = lyricsgenius.Genius(
        'v3s3cdBYSN4Kta3F2HDTmdqqNOckKEZTQA5O5Bhb5J-BV_L02jqPGCEnvpKsSrBz')
        
    df_with_lyrics['list_of_words'] = [i.split(' ') for i in df_with_lyrics['lyrics']]
    
    return df_with_lyrics